# Добро пожаловать в игру "Школа 2025"

Ниже представлен класс инициализации для игры, который затем будет использоваться при прохождении

In [11]:
class GameEngine:
    def __init__(self):
        self.status = 0
        self.counter = 0
        self.current_scene = "Start scene"
        self.choices_made = []
        self.player_name = ""

        self.scenes = {
            "Start scene": {
                "First choice": "First consequence",
                "Second choice": "Second consequence",
                "Third choice": "Third consequence"
            },
            "First consequence": {
                "First.First choice": "Early end 1",
                "First.Second choice": "Fourth consequence"
            },
            "Second consequence": {
                "Second.First choice": "Early end 2",
                "Second.Second choice": "Second consequence"
            },
            "Third consequence": {
                "Third.First choice": "Early end 3",
                "Third.Second choice": "Early end 4"
            },
            "Fourth consequence": {
                "Fourth.First choice": "Early end 5",
                "Fourth.Second choice": "Early end 6"
            }
        }

    def start(self):
        self.player_name = input("Введите имя игрока: ")
        self.handle_scene("Start scene")

    def handle_scene(self, scene_name):
        self.current_scene = scene_name
        self.counter += 1

        choices = self.scenes.get(scene_name, {})
        if not choices:
            print(f"\n🏁 Конец игры: {scene_name}")
            print(f"🔢 Вы прошли игру за {self.counter} шагов.")
            self.save_results()
            self.dfs()
            self.count_endings()
            return

        print(f"\n📍 Сцена: {scene_name}")
        print("Выберите действие:")
        options = list(choices.keys())
        for i, option in enumerate(options):
            print(f"{i + 1}: {option}")

        try:
            choice_index = int(input("Ваш выбор (номер): ")) - 1
            if 0 <= choice_index < len(options):
                choice_text = options[choice_index]
                self.choices_made.append(choice_text)
                next_scene = choices[choice_text]
                self.current_scene = next_scene
                self.check_next_scene()
            else:
                print("⚠ Неверный номер. Повторите ввод.")
                self.handle_scene(scene_name)
        except ValueError:
            print("⚠ Введите только число.")
            self.handle_scene(scene_name)

    def check_next_scene(self):
        if self.current_scene.startswith("Early end"):
            print(f"\n🏁 Конец игры: {self.current_scene}")
            print(f"🔢 Вы прошли игру за {self.counter} шагов.")
            self.save_results()
            self.dfs()
            self.count_endings()
        else:
            self.handle_scene(self.current_scene)

    def check_status(self):
        match self.status:
            case 0:
                self.start()
            case 1:
                self.first_scene()
            case 2:
                self.second_scene()
            case 3:
                self.third_scene()
            case 4:
                self.fourth_scene()

    def dfs(self):
        print("\n🧠 Анализ путей (DFS):")
        visited = set()
        min_steps = float('inf')

        def dfs_recursive(scene, steps):
            nonlocal min_steps
            if scene.startswith("Early end"):
                min_steps = min(min_steps, steps)
                return
            for next_scene in self.scenes.get(scene, {}).values():
                dfs_recursive(next_scene, steps + 1)

        dfs_recursive("Start scene", 0)

        print(f"🔍 Минимальное количество шагов до конца: {min_steps}")
        if self.counter > min_steps:
            print(f"Можно было пройти быстрее на {self.counter - min_steps} шаг(а/ов).")
        else:
            print("Вы прошли самым коротким путём!")

    def count_endings(self):
        print("\n🌲 Подсчёт всех возможных концовок...")

        endings = set()

        def recurse(scene):
            if scene.startswith("Early end"):
                endings.add(scene)
                return
            for next_scene in self.scenes.get(scene, {}).values():
                recurse(next_scene)

        recurse("Start scene")
        print(f"📦 Всего уникальных концовок: {len(endings)}")
        print("📌 Список концовок:")
        for e in sorted(endings):
            print(f" - {e}")

    def save_results(self):
        with open("results.txt", "a", encoding="utf-8") as f:
            f.write(f"👤 Игрок: {self.player_name}\n")
            f.write(f"📜 Сделанные выборы:\n")
            for i, choice in enumerate(self.choices_made, 1):
                f.write(f"  {i}. {choice}\n")
            f.write(f"🏁 Финал: {self.current_scene}\n")
            f.write(f"⭐ Опыт (шаги): {self.counter}\n")
            f.write(f"{'-'*40}\n")

        print("📝 Результаты сохранены в файл results.txt.")
